In [1]:
import h5py
import numpy as np

from ada import Assembly, Beam, Material, Part
from ada.fem import Bc, FemSet, Step
from ada.fem.io.mesh.recipes import create_beam_mesh
from ada.fem.utils import get_beam_end_nodes
from ada.materials.metals import CarbonSteel

from ada.fem.visualize import visualize_it

In [2]:
def fundamental_eigenfrequency(bm: Beam):
    L = bm.length
    E = bm.material.model.E
    rho = bm.material.model.rho
    I = bm.section.properties.Iy
    return (1 / (2 * np.pi)) * (3.5156 / L ** 2) * np.sqrt(E * I / rho)


bm = Beam("MyBeam", (0, 0.5, 0.5), (5, 0.5, 0.5), "IPE400", Material("S420", CarbonSteel("S420")))
bm

Beam("MyBeam", [0.0, 0.5, 0.5], [5.0, 0.5, 0.5], IPE400, S420)

In [3]:
f1 = fundamental_eigenfrequency(bm)
f2 = 6.268 * f1
f3 = 17.456 * f1
print(f"1: {f1}\n2: {f2}\n3: {f3}")

1: 2.0565050317553615
2: 12.890173539042605
3: 35.89835183432159


In [4]:
def make_fem(beam, assembly, part, mesh_type):
    create_beam_mesh(beam, part.fem, "shell") if mesh_type == "shell" else part.gmsh.mesh(0.1)
    fix_set = part.fem.add_set(FemSet("bc_nodes", get_beam_end_nodes(beam), "nset"))
    assembly.fem.add_bc(Bc("Fixed", fix_set, [1, 2, 3, 4, 5, 6]))
    assembly.fem.add_step(Step("Eigen", "eigenfrequency"))
   
p = Part("MyPart")
a = Assembly("MyAssembly") / [p / bm]
make_fem(bm, a, p, "beam")
res = a.to_fem("Cantilever_CA_EIG_bm", "code_aster", overwrite=True, execute=True)

f = h5py.File(res.results_file_path)
modes = f.get("CHA/modes___DEPL")

for mname, m in modes.items():
    mode = m.attrs["NDT"]
    freq = m.attrs["PDT"]

    print(mode, freq)

visualize_it(res.results_file_path)

Removing old files before copying new
Created a Code_Aster input deck at "C:\AibelProgs\ADA\scratch\Cantilever_CA_EIG_bm"
--------------------------------------------------------------------------------
starting CodeAsterAnalysis simulation "Cantilever_CA_EIG_bm"
Finished CodeAsterAnalysis simulation "Cantilever_CA_EIG_bm"
--------------------------------------------------------------------------------
1 4.672562038746128
2 6.170775817056764
3 18.518418261566026
4 22.898314280607366
5 29.282447999560382
6 30.884339083010413
7 43.28074337142302
8 55.71986436549283
9 68.21397245193279
10 80.77538180753837
11 81.99173251105687
12 93.41645386861136
13 106.14959670090452
14 118.98725920121504
15 131.94191888574105
16 143.5012936184222
17 145.02606180624946
18 158.25215287849218
19 160.67126415803756
20 171.63259460303757


AppLayout(children=(Scene(children=[Warp(input=(('modes___DEPL[0]_-_4.67256', 'X1'), ('modes___DEPL[0]_-_4.672…

In [5]:
p = Part("MyPart")
a = Assembly("MyAssembly") / [p / bm]
make_fem(bm, a, p, "shell")
res = a.to_fem("Cantilever_CA_EIG_sh", "code_aster", overwrite=True, execute=True)

f = h5py.File(res.results_file_path)
modes = f.get("CHA/modes___DEPL")

for mname, m in modes.items():
    mode = m.attrs["NDT"]
    freq = m.attrs["PDT"]

    print(mode, freq)

visualize_it(res.results_file_path)

AppLayout(children=(Scene(children=[Warp(factor=0.7, input=[('modes___DEPL[1]_-_10.6547', 'X1'), ('modes___DEP…